In [1]:
import sys
sys.path.insert(0, '/Users/base/Desktop/G-AI/CONCEPTS/MCP') # Adjust the path 

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
import sys


/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

#test to check if sub-process server is working
# !python /Users/base/Desktop/G-AI/CONCEPTS/MCP/mcp_server.py


In [ ]:
#run the server first in separate terminal: python mcp_server_stdio.py
#connects tot the MCP, keeps it as sub-process

client = MultiServerMCPClient(
    {
        "math": {
            "transport": "stdio",
            "command": sys.executable,         # e.g. 'python'
            "args": ["/Users/base/Desktop/G-AI/CONCEPTS/MCP/mcp_server_stdio.py"],        
        }
    }
)


In [11]:
tools = await client.get_tools()
tools


[StructuredTool(name='add', description='Adds two integers together.\nargs:\n    a: The first integer.\n    b: The second integer.\nreturns:\n    The sum of the two integers.', args_schema={'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}, metadata={'_meta': {'_fastmcp': {'tags': []}}}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x108075080>),
 StructuredTool(name='multiply_tool', description='Multiplies two integers together.', args_schema={'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}, metadata={'_meta': {'_fastmcp': {'tags': []}}}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x1080753a0>)]

In [12]:
# Groq model
from langchain_groq import ChatGroq
GROQ_MODEL = "llama-3.1-8b-instant"

# Initialize Groq-backed LLM for LangChain
model = ChatGroq(
    model=GROQ_MODEL,
    temperature=0.1,
    max_tokens=1000,
    timeout=30
)



In [13]:
agent = create_agent(
    model,  # or your preferred model identifier
    tools,
)


In [19]:
from pprint import pprint

result = await agent.ainvoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What is (3 + 5) * 12 using your tools?"
            }
        ]
    }
)

pprint(result)


{'messages': [HumanMessage(content='What is (3 + 5) * 12 using your tools?', additional_kwargs={}, response_metadata={}, id='a3c731a9-4447-4eb3-bff9-87b53b46c349'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'aej4s5xz4', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'add'}, 'type': 'function'}, {'id': '05chg62mk', 'function': {'arguments': '{"a":8,"b":12}', 'name': 'multiply_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 343, 'total_tokens': 380, 'completion_time': 0.052517127, 'prompt_time': 0.020354568, 'queue_time': 0.049278502, 'total_time': 0.072871695}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--da578abb-c6e7-4a1d-a5bd-9a5c90bdb4bb-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'aej4s5xz4', 'type': 'tool_call'}, {'na